In [ ]:
import plotly.express as px

In [ ]:
# #Data GeoJson Peta Indonesia
# from urllib.request import urlopen
# import json
# with urlopen('https://raw.githubusercontent.com/ans-4175/peta-indonesia-geojson/master/indonesia-prov.geojson') as response:
#     counties = json.load(response)

# counties["features"][2]

In [ ]:
#Buat function untuk ambil data covid dunia
import requests
def get_json(api_url):
    response = requests.get(api_url)
    if response.status_code == 200:
        return json.loads(response.content.decode('utf-8'))
    else:
        return None

In [ ]:
#Grep data hari ini
import datetime
import pandas as pd
import json
record_date = datetime.date.today().strftime('%Y-%m-%d')
covid_url = 'https://covid19-api.org/api/status?date='+record_date
df_covid_worldwide = pd.io.json.json_normalize(get_json(covid_url))

print(df_covid_worldwide.tail())

In [ ]:
#Merubah format tanggal di kolom last_update jadi format tgl hari ini
df_covid_worldwide['last_update'] = pd.to_datetime(df_covid_worldwide['last_update'], format='%Y-%m-%d')
df_covid_worldwide['last_update'] = df_covid_worldwide['last_update'].apply(lambda x: x.date())
df_covid_worldwide = df_covid_worldwide.rename(columns={'country':'alpha2'})
print(df_covid_worldwide.head(10))

In [ ]:
#Mengambil data countries
countries_url = 'https://covid19-api.org/api/countries'
df_countries = pd.io.json.json_normalize(get_json(countries_url))
df_countries = df_countries.rename(columns={'alpha3': 'country_code','name':'country_name'})[['country_name','country_code','alpha2']]

print(df_countries.head())

In [ ]:
#menggabungkan data covid dengan data countries
df_covid_denormalized = pd.merge(df_covid_worldwide, df_countries, on='alpha2')
print(df_covid_denormalized.head())

In [ ]:
#menambahkan kolom fatalitiy_ratio
df_covid_denormalized['fatality_ratio'] = df_covid_denormalized['deaths']/df_covid_denormalized['cases']

In [ ]:
#mengambil data 20 negara dengan top fatalitiy_ratio
df_top_20_fatality_rate = df_covid_denormalized.sort_values(by='fatality_ratio', ascending=False).head(20)
df_top_20_fatality_rate

In [ ]:
#membuat grafik histogram 20 negara dengan top fatality_ratio
fig = px.histogram(df_top_20_fatality_rate,x='country_name',y='fatality_ratio',color='country_name',height=1000)
fig.show()

# Data Persebaran Covid Di Dunia

In [ ]:
countries = []
for x in df_covid_worldwide.alpha2:
    countries.append(x)
    
i = 0
for country in countries:
    covid_timeline_url = 'https://covid19-api.org/api/timeline/'+country
    df_world_covid_timeline = pd.io.json.json_normalize(get_json(covid_timeline_url))
    df_world_covid_timeline['last_update'] = pd.to_datetime(df_world_covid_timeline['last_update'], format='%Y-%m-%d')
    df_world_covid_timeline['last_update'] = df_world_covid_timeline['last_update'].apply(lambda x: x.date())
    if i==0:
        df_world_covid_timeline_merged = df_world_covid_timeline
    else:
        df_world_covid_timeline_merged = df_world_covid_timeline.append(df_world_covid_timeline_merged, ignore_index=True)
    i=i+1

df_world_covid_timeline_merged = df_world_covid_timeline_merged.rename(columns={'country':'alpha2'})
df_world_covid_timeline_denormalized = pd.merge(df_world_covid_timeline_merged , df_countries, on='alpha2')
df_world_covid_timeline_denormalized = df_world_covid_timeline_denormalized[(df_world_covid_timeline_denormalized['last_update'] >= datetime.date(2020, 3, 1))]
df_world_covid_timeline_denormalized.last_update = df_world_covid_timeline_denormalized.last_update.astype('str')
df_world_covid_timeline_denormalized = df_world_covid_timeline_denormalized.sort_values(by='last_update')
print(df_world_covid_timeline_merged.head())

In [ ]:
df_world_covid_timeline_denormalized.country_code

In [ ]:
# px.choropleth(df_world_covid_timeline_denormalized,locations='country_code',color='cases',hover_name='country_name',
#               height=700,color_continuous_scale='algae',scope='asia',
#               projection='natural earth',hover_data=df_world_covid_timeline_denormalized[['cases','deaths','recovered']])

In [ ]:
px.choropleth(df_world_covid_timeline_denormalized,locations='country_code',color='cases',hover_name='country_name',
              animation_frame='last_update',height=700,
              projection='orthographic',hover_data=df_world_covid_timeline_denormalized[['cases','deaths','recovered']])

In [ ]:
px.choropleth(df_world_covid_timeline_denormalized,locations='country_code',color='cases',hover_name='country_name',
              animation_frame='last_update',color_continuous_scale='algae',height=700,
              projection='natural earth',hover_data=df_world_covid_timeline_denormalized[['cases','deaths','recovered']])